In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath('../..'))
from experiments.iter2.util.importer_LSTM import *
GRAPH_PATH = 'graph_saves/ex1_graphs/ex1_'
MODEL_PATH = 'model_saves/ex1_models/ex1_model'
nist_path = "../../src/data_preprocess/dataset/NIST_cleaned.csv"
SEED = 1337
SEED_COUNT = 15
from experiments.iter1.util.evaluate import evaluate_model

In [ ]:
if not os.path.exists('model_saves/ex1_models'):
    os.mkdir('model_saves/ex1_models')
if not os.path.exists('graph_saves/ex1_graphs'):
    os.mkdir('graph_saves/ex1_graphs')


for i in range(SEED_COUNT):
    seed_everything(SEED, workers=True)

    assert time_horizon > 0, "Time horizon must be a positive integer"
        
    df = pd.read_csv(nist_path)


    cleaner = TempCleaner(clean_pow_low, clean_in_low, clean_in_high, clean_out_low, clean_out_high, clean_delta_temp)
    splitter = DaySplitter(TIMESTAMP, POWER ,train_days, val_days, test_days)
        
    model = LSTM(hidden_size, num_layers, input_size, time_horizon, dropout)
    trainer = TrainerWrapper(L.Trainer, 
                            max_epochs=num_epochs, 
                            callbacks=[EarlyStopping(monitor='val_loss', min_delta=0.0, patience=3, verbose=False, mode='min', strict=True)], 
                            gradient_clip_val=gradient_clipping, deterministic=True)
    optimizer = OptimizerWrapper(optim.Adam, model, lr=learning_rate)

    model = DeterministicPipeline.Builder() \
        .add_data(df) \
        .set_cleaner(cleaner) \
        .set_normalizer_class(MinMaxNormalizer) \
        .set_splitter(splitter) \
        .set_sequencer_class(AllTimeSequencer) \
        .set_target_column(TARGET_COLUMN) \
        .set_model(model) \
        .set_optimizer(optimizer) \
        .set_batch_size(batch_size) \
        .set_seq_len(seq_len) \
        .set_worker_num(NUM_WORKERS) \
        .set_error(NRMSE) \
        .set_train_error(RMSE) \
        .set_trainer(trainer) \
        .set_use_tuner(True) \
        .build()

    model.fit(),
    plot_loss([model.get_validation_loss(), model.get_training_loss()], GRAPH_PATH + "LOSS_" + str(SEED), ["Validation loss", "Training loss"])
    torch.save(model.state_dict(), MODEL_PATH + "_" + str(SEED) + ".pth")

    SEED += 10

In [ ]:
from experiments.iter1.util.evaluate import evaluate_model

df = pd.read_csv(nist_path)

cleaner = TempCleaner(clean_pow_low, clean_in_low, clean_in_high, clean_out_low, clean_out_high, clean_delta_temp)
splitter = DaySplitter(TIMESTAMP, POWER ,train_days, val_days, test_days)
    
model = LSTM(hidden_size, num_layers, input_size, time_horizon, dropout)
trainer = TrainerWrapper(L.Trainer, 
                         max_epochs=num_epochs, 
                         callbacks=[EarlyStopping(monitor='val_loss', min_delta=0.0, patience=3, verbose=False, mode='min', strict=True)], 
                         gradient_clip_val=gradient_clipping, deterministic=True)
optimizer = OptimizerWrapper(optim.Adam, model, lr=learning_rate)

model = DeterministicPipeline.Builder() \
    .add_data(df) \
    .set_cleaner(cleaner) \
    .set_normalizer_class(MinMaxNormalizer) \
    .set_splitter(splitter) \
    .set_sequencer_class(AllTimeSequencer) \
    .set_target_column(TARGET_COLUMN) \
    .set_model(model) \
    .set_optimizer(optimizer) \
    .set_batch_size(batch_size) \
    .set_seq_len(seq_len) \
    .set_worker_num(NUM_WORKERS) \
    .set_error(NRMSE) \
    .set_train_error(RMSE) \
    .set_trainer(trainer) \
    .build()

model.load_state_dict(torch.load(MODEL_PATH + "_seed1.pth", weights_only=True))
model.test()

evaluate_model(model, df, splitter, cleaner, TIMESTAMP, POWER, on_limit_w, off_limit_w, consecutive_points, seq_len, time_horizon, TARGET_COLUMN, error, temp_boundary, None)
